In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns

In [2]:
df = pd.read_csv('data6.csv')

C:\Users\nicol\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Coverting to datetime format.
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y") 

# First, some data insights or Bussiness Analysis on target value.

###  Top 5 most sold liqours by liters. 2012 - 2017

In [4]:
print( df.groupby(['Category Name'])['Volume Sold (Liters)'].sum().sort_values(ascending=False)[:5] )

Category Name
VODKA 80 PROOF                      17797396.77
CANADIAN WHISKIES                   11788955.90
SPICED RUM                           6535656.32
BLENDED WHISKIES                     3943552.25
PUERTO RICO & VIRGIN ISLANDS RUM     3889666.92
Name: Volume Sold (Liters), dtype: float64


### Top 5 least sold liqours by liters. 2012 - 2017

In [5]:
print( df.groupby(['Category Name'])['Volume Sold (Liters)'].sum().sort_values(ascending= True)[:5] )

Category Name
American Gins                            5.00
Temporary &  Specialty Packages         10.50
HIGH PROOF BEER - AMERICAN              40.50
Imported Distilled Spirit Specialty    149.67
IMPORTED VODKA - CHERRY                222.00
Name: Volume Sold (Liters), dtype: float64


### The years when the most Liters of liquour were sold.

In [6]:
print ( df.groupby(df['Date'].dt.year)['Volume Sold (Liters)'].sum().sort_values(ascending=False))


Date
2015    19617702.47
2014    19157612.51
2012    18755279.42
2013    18576450.22
2016    14508093.43
2017     3680936.84
Name: Volume Sold (Liters), dtype: float64


### Top cities 5 where the most liters are sold.

In [7]:
df['City'].value_counts()[:5]

DES MOINES      811086
CEDAR RAPIDS    597416
DAVENPORT       411104
Des Moines      307692
WATERLOO        289538
Name: City, dtype: int64

### Most preferable Bottle capacity bought by customers ( in liters)

In [8]:
(df['Bottle Volume (ml)']/1000).value_counts().head()

0.750    5719505
1.750    2550403
1.000    1787513
0.375    1224058
0.500     523506
Name: Bottle Volume (ml), dtype: int64

# Preparing the sample.

In [9]:
# Text data. Either description or helping data for certain features. Just keeping the categorical data represented by numbers.
df = df.drop(['Invoice/Item Number', 'Store Name', 'Address', 'Store Location','Item Description', 
             'County', 'Category Name', 'Vendor Name',  'Item Description',  'Volume Sold (Gallons)', ],axis = 1) 

# Mixed datatypes.Replacing the strings
df['Zip Code'] = df['Zip Code'].replace('712-2', 7122)   

# Converting to numerical values (ignoring erros for nan values, replaced below.)
df['Zip Code'] = df['Zip Code'].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))  


# A Normal distribution of the series + null values:(10-2k,+ max 8k) 
# + sample is a big dataset = dropping the rows with null values. 
# Exception: the 'County Number': left skewed distributted ( mode > median > mean), could be replaced by .median() BUT
# BUT since the County Number is just numerical represented by categorical County ->
#  -> I drop it off JUST TO keep the accuracy and certainty of the data.
df = df.dropna() 

In [10]:
# Converting the ML to L
df['Bottle Volume (ml)'] =  df['Bottle Volume (ml)']/1000 
df.rename(columns = {'Bottle Volume (ml)' : 'Bottle Volume'}, inplace = True)

In [11]:
 # Removing  $ from the given series. Coverts to float.
for col in [ "State Bottle Retail", "Sale (Dollars)", "State Bottle Cost"]:
    df[col]= df[col].apply(lambda x : float(x[1:]) )

## I have observed some ambiguous data. 
#### Either the Cost and Retail  values are missing and the Sale features are present
#### Or the Sale is 0 but the Cost and Retail prices exist. 
##### Additionally to that, there is always shown  a value for the target column, Sold Volume. despite of 0s present in the cost features

In [12]:
# Total numbers of ambiguous data.
df['Sale (Dollars)'].isin( [0] ).sum()

4289

In [13]:
# Representation of ambiguous data.
df[ df['Sale (Dollars)'].isin( [0] ) == True ]

,Date,Store Number,City,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters)
230,2012-02-15,4319,HUMBOLDT,50548.0,46.0,1051110.0,434.0,55086,12,0.75,0.00,0.00,12,0.0,9.00
4459,2012-02-28,3808,LA PORTE CITY,50651.0,7.0,1062310.0,434.0,44657,12,1.00,0.00,0.00,2,0.0,2.00
5367,2012-02-28,2619,WEST DES MOINES,50266.0,77.0,1081300.0,434.0,81406,12,0.75,0.00,0.00,3,0.0,2.25
16190,2012-02-06,2228,WINTERSET,50273.0,61.0,1051100.0,434.0,54446,12,0.75,0.00,0.00,12,0.0,9.00
17436,2012-02-23,2647,CEDAR RAPIDS,52411.0,57.0,1081330.0,434.0,84456,12,0.75,0.00,0.00,4,0.0,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12580779,2017-10-31,2594,Sioux City,51106.0,97.0,1032200.0,395.0,35964,12,0.75,9.96,14.94,1,0.0,0.75
12581693,2017-10-31,2623,Sioux City,51104.0,97.0,1082000.0,192.0,65251,5,1.20,29.53,44.30,1,0.0,1.20
12581785,2017-10-31,3814,West Des Moines,50266.0,25.0,1012300.0,370.0,5036,12,0.75,23.48,35.22,1,0.0,0.75
12581926,2017-10-31,2190,Des Moines,50314.0,77.0,1081400.0,65.0,82898,12,1.00,7.87,11.81,1,0.0,1.00


In [14]:
# Dropping the ambigous data.
df =  df[ df['Sale (Dollars)'].isin( [0] ) == False ]

In [15]:
# Resetting the indexes and droppping the automated  assigned column.
df.reset_index(inplace = True)
df = df.drop('index',axis= 1 )

##### We could manipulate the features in cost formulas.  Gross margin, Margin,  Contribution Margin, Markup etc. 
##### Next, I will select a few new columns to be added to avoid multicollinearity, the new columns that a little better correlation values.

In [16]:
# Dropping the date since it just helped for data exploration.
df = df.drop ('Date',axis =1 )  

In [17]:
# Amount of money a company retains after incurring the direct costs associated with producing the goods it sells  
df['Gross Margin'] = df['Sale (Dollars)'] -  df['State Bottle Cost']    

# Price per liter for each bottle.
df['Price per Liter'] = df['State Bottle Retail']/ df['Bottle Volume'] 

# Creating the total profit of bottles sold by multiplying the Margin or Profit with bottles sold.
df['Total Margin'] =  ( df['State Bottle Retail'] -  df['State Bottle Cost']  ) * df['Bottles Sold']  

In [18]:
 df['Price per Liter'].isin( [np.inf] ).sum()

10

In [19]:
# Dropping off the 10 rows where the infinity is met. 
df = df [ df['Price per Liter'].isin( [np.inf, -np.inf] ) == False ] 

In [20]:
# Shifting the target column to the left of the sample.
temp  =df['Volume Sold (Liters)'].copy()
df.pop('Volume Sold (Liters)') 
df = pd.concat([df,temp], axis =1)

In [21]:
# Dropping off the categorical City
df = df.drop('City', axis =1 )

#####  Methods for correlation:  spearman corr > pearson corr : meaning that is a  monotonic trend. It means that the variable consistently increases (decreases) but the trend may or may not be linear. 

In [22]:
df.corr(method = 'spearman')

,Store Number,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume,State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Gross Margin,Price per Liter,Total Margin,Volume Sold (Liters)
Store Number,1.000000,0.001382,0.031967,-0.029123,-0.000729,-0.025163,0.082223,-0.108096,-0.082169,-0.082578,-0.047463,-0.062212,-0.059348,0.006912,-0.084556,-0.088432
Zip Code,0.001382,1.000000,-0.089757,0.008369,-0.000109,0.016724,0.024808,-0.017408,-0.027594,-0.027710,0.005718,-0.005966,-0.003052,-0.007966,-0.008934,-0.003149
County Number,0.031967,-0.089757,1.000000,-0.000274,-0.002149,0.003369,0.014420,-0.034772,0.003794,0.003760,0.013774,0.025832,0.025071,0.032253,0.015999,-0.004365
Category,-0.029123,0.008369,-0.000274,1.000000,0.182002,0.699988,0.026344,-0.016169,-0.029222,-0.029303,-0.029322,-0.046221,-0.041578,0.009135,-0.043394,-0.032841
Vendor Number,-0.000729,-0.000109,-0.002149,0.182002,1.000000,0.244791,-0.038065,0.021663,-0.046137,-0.047047,0.006053,-0.013927,-0.013701,-0.075695,-0.018318,0.018278
Item Number,-0.025163,0.016724,0.003369,0.699988,0.244791,1.000000,0.069849,-0.068232,-0.051502,-0.051896,-0.029979,-0.063498,-0.057764,0.026310,-0.055222,-0.058069
Pack,0.082223,0.024808,0.014420,0.026344,-0.038065,0.069849,1.000000,-0.728678,-0.495070,-0.496068,0.094690,-0.199972,-0.153997,0.194169,-0.186028,-0.281860
Bottle Volume,-0.108096,-0.017408,-0.034772,-0.016169,0.021663,-0.068232,-0.728678,1.000000,0.420862,0.421657,0.059370,0.315635,0.283655,-0.427535,0.278558,0.511002
State Bottle Cost,-0.082169,-0.027594,0.003794,-0.029222,-0.046137,-0.051502,-0.495070,0.420862,1.000000,0.999780,-0.137946,0.371442,0.289395,0.594947,0.368624,0.082489
State Bottle Retail,-0.082578,-0.027710,0.003760,-0.029303,-0.047047,-0.051896,-0.496068,0.421657,0.999780,1.000000,-0.137609,0.371677,0.289668,0.594177,0.369315,0.083156


In [23]:
df = df.drop([ 
             # Dropping the categorical data with low correlations
             'Zip Code',   'County Number', 'Store Number', 'Category',   'Vendor Number', 'Item Number',
    
              # Dropping the Sold feature since it has a 0.9 corr with the target value. 
              # Actually, it  has the same meaning as the target value.
              'Bottles Sold',  
              
              # Dropping the Cost Bussiness Formulas since the Total Margin has a little better corr compared with other features. 
              # And keeping them means Multicollinearity since it deduces from each other.
             'State Bottle Retail',   'Sale (Dollars)', 'Gross Margin', 'State Bottle Cost', 
              
              # Dropping because of multicollinearity
              'Pack', 'Price per Liter'
             ], axis=1)

In [24]:
df.corr(method = 'spearman')

,Bottle Volume,Total Margin,Volume Sold (Liters)
Bottle Volume,1.000000,0.278558,0.511002
Total Margin,0.278558,1.000000,0.857676
Volume Sold (Liters),0.511002,0.857676,1.000000


In [25]:
df

,Bottle Volume,Total Margin,Volume Sold (Liters)
0,1.50,34.86,9.0
1,0.75,39.16,3.0
2,1.50,5.81,1.5
3,6.00,49.50,6.0
4,1.50,34.86,9.0
...,...,...,...
12499577,0.80,12.00,2.4
12499578,0.75,27.48,1.5
12499579,0.75,24.98,1.5
12499580,0.75,22.50,1.5


In [27]:
# Saving and proceeding to the next step.
df.to_csv('df.csv', index=False)  